#Importing Libraries

In [ ]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import re
import pandas as pd
import numpy as np
import os



# Importing libraries
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam


import re
import pickle
import numpy as np
import pandas as pd
# Plot libraries
import seaborn as sns
#from wordcloud import WordCloud
import matplotlib.pyplot as plt

Loading the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('/content/drive/MyDrive/Dataset1_after_augmentation.csv', encoding=DATASET_ENCODING)
#dataset = pd.read_csv('/content/Dataset1_after_augmentation.csv')
dataset.head()

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
dataset2 = pd.read_csv('/content/drive/MyDrive/Dataset2_after_augmentation.csv', encoding=DATASET_ENCODING)
dataset2.head()

Ploting the Data --> Sentiment

In [ ]:
ax = dataset.groupby('Label').count().plot(kind='bar', title='Distribution of data', legend=True)
ax = ax.set_xticklabels(['0','1','2'], rotation=0)

In [ ]:
ax = dataset2.groupby('Label').count().plot(kind='bar', title='Distribution of data', legend=True)
ax = ax.set_xticklabels(['0','1','2'], rotation=0)

Preprocessing the Dataset

In [ ]:
# Reading contractions.csv and storing it as a dict.
contractions = pd.read_csv('/content/drive/MyDrive/preprocessing_contractions.csv', index_col='Contraction')
contractions.index = contractions.index.str.lower()
contractions.Meaning = contractions.Meaning.str.lower()
contractions_dict = contractions.to_dict()['Meaning']

# Defining regex patterns.
urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"
userPattern       = '@[^\s]+'
hashtagPattern    = '#[^\s]+'
alphaPattern      = "[^a-z0-9<>]"
sequencePattern   = r"(.)\1\1+"
seqReplacePattern = r"\1\1"

# Defining regex for emojis
smileemoji        = r"[8:=;]['`\-]?[)d]+"
sademoji          = r"[8:=;]['`\-]?\(+"
neutralemoji      = r"[8:=;]['`\-]?[\/|l*]"
lolemoji          = r"[8:=;]['`\-]?p+"

def preprocess_apply(tweet):

    tweet = tweet.lower()

    # Replace all URls with '<url>'
    tweet = re.sub(urlPattern,'<url>',tweet)
    # Replace @USERNAME to '<user>'.
    tweet = re.sub(userPattern,'<user>', tweet)
    
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

    # Replace all emojis.
    tweet = re.sub(r'<3', '<heart>', tweet)
    tweet = re.sub(smileemoji, '<smile>', tweet)
    tweet = re.sub(sademoji, '<sadface>', tweet)
    tweet = re.sub(neutralemoji, '<neutralface>', tweet)
    tweet = re.sub(lolemoji, '<lolface>', tweet)

    for contraction, replacement in contractions_dict.items():
        tweet = tweet.replace(contraction, replacement)

    # Remove non-alphanumeric and symbols
    tweet = re.sub(alphaPattern, ' ', tweet)

    # Adding space on either side of '/' to seperate words (After replacing URLS).
    tweet = re.sub(r'/', ' / ', tweet)
    return tweet

In [ ]:
dataset['processed_text'] = dataset['Text'].apply(preprocess_apply)

In [ ]:
dataset.head()

In [ ]:
dataset2['processed_text'] = dataset2['Text'].apply(preprocess_apply)

In [ ]:
dataset2.head()

Showing the Preprocess the Data

In [ ]:
count=0
for row in dataset.itertuples():
    print("Text:", row[2])
    print("Processed:", row[3])
    count+=1
    if count>10:
        break

In [ ]:
count=0
for row in dataset2.itertuples():
    print("Text:", row[2])
    print("Processed:", row[3])
    count+=1
    if count>10:
        break

Word2Vec

In [ ]:
import spacy
!python -m spacy download en_core_web_lg
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
x = 'king man woman'
doc =nlp(x)
for token1 in doc:
    for token2 in doc:
        print(token1.text, token2.text, token1.similarity(token2))

In [ ]:
def get_vec(x):
    doc = nlp(x)
    vec = doc.vector
    return vec

In [ ]:
dataset['vec']= dataset['processed_text'].apply(lambda x: get_vec(x))

In [ ]:
dataset.head()


Spliting the dataset1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#X_data, Y_data = np.array(dataset['vec']), np.array(dataset['Label'])
X_data, Y_data = dataset['Text'], dataset['Label']
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size = 0.02, random_state = 0)
print('Data Split done.')

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

Splitting the dataset2


In [ ]:
X_data2, Y_data2 = dataset2['Text'], dataset2['Label']
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_data2, Y_data2, test_size = 0.02, random_state = 0)
print('Data Split done.')

In [ ]:
X_train2.shape, X_test2.shape, Y_train2.shape, Y_test2.shape

#CNN: Dataset1

In [ ]:

#####----- Before running this model run the train-test split portion once -----#####

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
Xcnn_train = tokenizer.texts_to_sequences(X_train)
Xcnn_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1  
print(X_train[1])
print(Xcnn_train[1]) 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 100
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :]) 

In [ ]:
from keras.models import Sequential
from keras import layers 
embedding_dim = 100
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 3, activation='relu'))
textcnnmodel.add(layers.GlobalMaxPooling1D())
textcnnmodel.add(layers.Dense(2, activation='relu'))
textcnnmodel.add(layers.Dense(1, activation='sigmoid'))
textcnnmodel.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])
textcnnmodel.summary()

In [ ]:
#original code
textcnnmodel.fit(Xcnn_train, Y_train,
                     epochs=100,
                     verbose=False,
                     validation_data=(Xcnn_test, Y_test),
                     batch_size=128)
loss, accuracy = textcnnmodel.evaluate(Xcnn_train, Y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy)) 

In [ ]:
textcnnmodel.save('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/saved_models/CNN.hdf5')

#CNN: Dataset2

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train2)
Xcnn2_train = tokenizer.texts_to_sequences(X_train2)
Xcnn2_test = tokenizer.texts_to_sequences(X_test2)
vocab_size = len(tokenizer.word_index) + 1  
print(X_train2[1])
print(Xcnn2_train[1]) 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 100
Xcnn2_train = pad_sequences(Xcnn2_train, padding='post', maxlen=maxlen)
Xcnn2_test = pad_sequences(Xcnn2_test, padding='post', maxlen=maxlen)
print(Xcnn2_train[0, :]) 

In [ ]:
from keras.models import Sequential
from keras import layers 
embedding_dim = 100
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 3, activation='relu'))
textcnnmodel.add(layers.GlobalMaxPooling1D())
textcnnmodel.add(layers.Dense(10, activation='relu'))
textcnnmodel.add(layers.Dense(1, activation='sigmoid'))
textcnnmodel.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])
textcnnmodel.summary()

In [ ]:
textcnnmodel.fit(Xcnn2_train, Y_train2,
                     epochs=100,
                     verbose=False,
                     validation_data=(Xcnn2_test, Y_test2),
                     batch_size=10)
loss, accuracy = textcnnmodel.evaluate(Xcnn2_train, Y_train2, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = textcnnmodel.evaluate(Xcnn2_test, Y_test2, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy)) 

In [ ]:
textcnnmodel.save('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/saved_models/CNN.hdf5')

#BiLSTM: dataset1

In [ ]:
#X_data, Y_data = np.array(dataset['vec']), np.array(dataset['Label'])
X_data, Y_data = dataset['Text'], dataset['Label']
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size = 0.02, random_state = 0)
print('Data Split done.')
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

X_data2, Y_data2 = dataset2['Text'], dataset2['Label']
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_data2, Y_data2, test_size = 0.02, random_state = 0)
print('Data Split done.')
X_train2.shape, X_test2.shape, Y_train2.shape, Y_test2.shape

In [ ]:

#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 60
vocab_length = 60000
Embedding_dimensions = 100

vocab_length = 60000

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test  = pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=input_length)

print("X_train.shape:", X_train.shape)
print("X_test.shape :", X_test.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

In [ ]:
history = training_model.fit(
    X_train, Y_train,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/saved_models/BiLSTM.hdf5')

#BiLSTM: Dataset2

In [ ]:
#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 60
vocab_length = 60000
Embedding_dimensions = 100

vocab_length = 60000

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data2)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train2 = pad_sequences(tokenizer.texts_to_sequences(X_train2), maxlen=input_length)
X_test2  = pad_sequences(tokenizer.texts_to_sequences(X_test2) , maxlen=input_length)

print("X_train2.shape:", X_train2.shape)
print("X_test2.shape :", X_test2.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model_BiLSTM = getModel()
training_model_BiLSTM.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

In [ ]:
history = training_model_BiLSTM.fit(
    X_train2, Y_train2,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/saved_models/BiLSTM.hdf5')

#LSTM - Dataset1

In [ ]:

#####----- Before running this model run the train-test split portion once -----#####

# Define parameter
max_len = 50 
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # out of vocabulary token
vocab_size = 500
embedding_dim = 100
tokenizer = Tokenizer(num_words = vocab_size, 
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)

In [ ]:
# Get the word_index
word_index = tokenizer.word_index
total_words = len(word_index)
total_words

In [ ]:
training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences,
                                maxlen = max_len,
                                padding = padding_type,
                                truncating = trunc_type)

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences,
                               maxlen = max_len,
                               padding = padding_type,
                               truncating = trunc_type)

In [ ]:
import tensorflow as tf
n_lstm = 128
drop_lstm = 0.2
# Define LSTM Model 
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model1.add(tf.keras.layers.SpatialDropout1D(drop_lstm))
model1.add(LSTM(n_lstm, return_sequences=False))
model1.add(Dropout(drop_lstm))
model1.add(Dense(1, activation='sigmoid'))
model1.summary()

In [ ]:
model1.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [ ]:
epochs = 100
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model1.fit(training_padded,
                     Y_train,
                     epochs=epochs, 
                     validation_data=(testing_padded, Y_test),
                     callbacks =[early_stop],
                     verbose=2)

In [ ]:
model1.save('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/saved_models/LSTM.hdf5')

#LSTM: Dataset2

In [ ]:
#####----- Before running this model run the train-test split portion once -----#####

# Define parameter
max_len = 50 
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # out of vocabulary token
vocab_size = 500
embedding_dim = 100
tokenizer = Tokenizer(num_words = vocab_size, 
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)

In [ ]:
# Get the word_index
word_index = tokenizer.word_index
total_words = len(word_index)
total_words

In [ ]:
training_sequences2 = tokenizer.texts_to_sequences(X_train2)
training_padded2 = pad_sequences(training_sequences,
                                maxlen = max_len,
                                padding = padding_type,
                                truncating = trunc_type)

In [ ]:
testing_sequences2 = tokenizer.texts_to_sequences(X_test2)
testing_padded2 = pad_sequences(testing_sequences,
                               maxlen = max_len,
                               padding = padding_type,
                               truncating = trunc_type)

In [ ]:
n_lstm = 128
drop_lstm = 0.2
# Define LSTM Model 
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model1.add(tf.keras.layers.SpatialDropout1D(drop_lstm))
model1.add(LSTM(n_lstm, return_sequences=False))
model1.add(Dropout(drop_lstm))
model1.add(Dense(1, activation='sigmoid'))
model1.summary()

In [ ]:
model1.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [ ]:
epochs = 100
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model1.fit(training_padded2,
                     Y_train2,
                     epochs=epochs, 
                     validation_data=(testing_padded2, Y_test2),
                     callbacks =[early_stop],
                     verbose=2)

In [ ]:
model1.save('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/saved_models/LSTM.hdf5')


#Ensemble Model: Dataset1

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score
import tensorflow as tf

tf.keras.models.load_model('/content/CNN_aug_1.h5')
model1 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/saved_models/CNN.hdf5')
model2 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/saved_models/LSTM.hdf5')
model3 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/saved_models/BiLSTM.hdf5')

In [ ]:
models = [model1, model2, model3]

preds = [model.predict(X_test) for model in models]
preds=np.array(preds)
summed = np.sum(preds, axis=0)

# argmax across classes
ensemble_prediction = np.argmax(summed, axis=1)

In [ ]:
prediction1 = model1.predict(X_test) 
prediction2 = model2.predict(X_test) 
prediction3 = model3.predict(X_test) 

accuracy1 = accuracy_score(Y_test, prediction1)
accuracy2 = accuracy_score(Y_test, prediction2)
accuracy3 = accuracy_score(Y_test, prediction3)
ensemble_accuracy = accuracy_score(Y_test, ensemble_prediction)

print('Accuracy Score for model1 = ', accuracy1)
print('Accuracy Score for model2 = ', accuracy2)
print('Accuracy Score for model3 = ', accuracy3)
print('Accuracy Score for average ensemble = ', ensemble_accuracy)

from sklearn.metrics import classification_report
# Print the evaluation metrics for the dataset.
# print(classification_report(Y_test, preds))
print(classification_report(Y_test, ensemble_accuracy))

#Ensemble Model: Dataset2

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

model1 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/saved_models/CNN.hdf5')
model2 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/saved_models/LSTM.hdf5')
model3 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/saved_models/BiLSTM.hdf5')

In [ ]:
models = [model1, model2, model3]

preds = [model.predict(X_test) for model in models]
preds=np.array(preds)
summed = np.sum(preds, axis=0)

# argmax across classes
ensemble_prediction = np.argmax(summed, axis=1)

In [ ]:
prediction1 = model1.predict(X_test2) 
prediction2 = model2.predict(X_test2) 
prediction3 = model3.predict(X_test2) 

accuracy1 = accuracy_score(Y_test2, prediction1)
accuracy2 = accuracy_score(Y_test2, prediction2)
accuracy3 = accuracy_score(Y_test2, prediction3)
ensemble_accuracy = accuracy_score(Y_test2, ensemble_prediction)

print('Accuracy Score for model1 = ', accuracy1)
print('Accuracy Score for model2 = ', accuracy2)
print('Accuracy Score for model3 = ', accuracy3)
print('Accuracy Score for average ensemble = ', ensemble_accuracy)

from sklearn.metrics import classification_report
# Print the evaluation metrics for the dataset.
# print(classification_report(Y_test, preds))
print(classification_report(Y_test2, ensemble_accuracy))

#Baseline BiLSTM ensemble model: Dataset 1

In [ ]:

#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 200
vocab_length = 60000
Embedding_dimensions = 100

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test  = pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=input_length)

print("X_train.shape:", X_train.shape)
print("X_test.shape :", X_test.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

history = training_model.fit(
    X_train, Y_train,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/BiLSTM i1,i2,i3/BiLSTM i1.hdf5')

In [ ]:

#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 600
vocab_length = 60000
Embedding_dimensions = 100

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test  = pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=input_length)

print("X_train.shape:", X_train.shape)
print("X_test.shape :", X_test.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

history = training_model.fit(
    X_train, Y_train,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/BiLSTM i1,i2,i3/BiLSTM i2.hdf5')

In [ ]:

#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 60
vocab_length = 60000
Embedding_dimensions = 100

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test  = pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=input_length)

print("X_train.shape:", X_train.shape)
print("X_test.shape :", X_test.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

In [ ]:
history = training_model.fit(
    X_train, Y_train,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/BiLSTM i1,i2,i3/BiLSTM i3.hdf5')

#BiLSTM Ensemble Model: Dataset 1

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

model1 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/BiLSTM i1,i2,i3/BiLSTM i1.hdf5')
model2 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/BiLSTM i1,i2,i3/BiLSTM i2.hdf5')
model3 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset1_aug files/BiLSTM i1,i2,i3/BiLSTM i3.hdf5')

In [ ]:
models = [model1, model2, model3]

preds = [model.predict(X_test) for model in models]
preds=np.array(preds)
summed = np.sum(preds, axis=0)

# argmax across classes
ensemble_prediction = np.argmax(summed, axis=1)

In [ ]:
prediction1 = model1.predict(X_test) 
prediction2 = model2.predict(X_test) 
prediction3 = model3.predict(X_test) 

accuracy1 = accuracy_score(Y_test, prediction1)
accuracy2 = accuracy_score(Y_test, prediction2)
accuracy3 = accuracy_score(Y_test, prediction3)
ensemble_accuracy = accuracy_score(Y_test, ensemble_prediction)

print('Accuracy Score for model1 = ', accuracy1)
print('Accuracy Score for model2 = ', accuracy2)
print('Accuracy Score for model3 = ', accuracy3)
print('Accuracy Score for average ensemble = ', ensemble_accuracy)

from sklearn.metrics import classification_report
# Print the evaluation metrics for the dataset.
# print(classification_report(Y_test, preds))
print(classification_report(Y_test, ensemble_accuracy))

#Baseline BiLSTM ensemble model: Dataset 2

In [ ]:
#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 200
vocab_length = 60000
Embedding_dimensions = 100

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data2)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train2 = pad_sequences(tokenizer.texts_to_sequences(X_train2), maxlen=input_length)
X_test2  = pad_sequences(tokenizer.texts_to_sequences(X_test2) , maxlen=input_length)

print("X_train.shape:", X_train2.shape)
print("X_test.shape :", X_test2.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

history = training_model.fit(
    X_train2, Y_train2,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/BiLSTM i1,i2,i3/BiLSTM i1.hdf5')

In [ ]:
#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 600
vocab_length = 60000
Embedding_dimensions = 100

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train2 = pad_sequences(tokenizer.texts_to_sequences(X_train2), maxlen=input_length)
X_test2  = pad_sequences(tokenizer.texts_to_sequences(X_test2) , maxlen=input_length)

print("X_train2.shape:", X_train2.shape)
print("X_test2.shape :", X_test2.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

history = training_model.fit(
    X_train2, Y_train2,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/BiLSTM i1,i2,i3/BiLSTM i2.hdf5')

In [ ]:
#####----- Before running this model run the train-test split portion once -----#####

from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding
input_length = 60
vocab_length = 60000
Embedding_dimensions = 100

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

X_train2 = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test2  = pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=input_length)

print("X_train2.shape:", X_train2.shape)
print("X_test2.shape :", X_test2.shape)

embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

In [ ]:
history = training_model.fit(
    X_train2, Y_train2,
    batch_size=1024,
    epochs=100,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
training_model.save('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/BiLSTM i1,i2,i3/BiLSTM i3.hdf5')

#BiLSTM Ensemble Model: dataset 2

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score

model1 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/BiLSTM i1,i2,i3/BiLSTM i1.hdf5')
model2 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/BiLSTM i1,i2,i3/BiLSTM i2.hdf5')
model3 = load_model('/content/drive/MyDrive/Thesis_Coding/Dataset2_aug files/BiLSTM i1,i2,i3/BiLSTM i3.hdf5')

In [ ]:
models = [model1, model2, model3]

preds = [model.predict(X_test2) for model in models]
preds=np.array(preds)
summed = np.sum(preds, axis=0)

# argmax across classes
ensemble_prediction = np.argmax(summed, axis=1)

In [ ]:
prediction1 = model1.predict(X_test2) 
prediction2 = model2.predict(X_test2) 
prediction3 = model3.predict(X_test2) 

accuracy1 = accuracy_score(Y_test, prediction1)
accuracy2 = accuracy_score(Y_test, prediction2)
accuracy3 = accuracy_score(Y_test, prediction3)
ensemble_accuracy = accuracy_score(Y_test, ensemble_prediction)

print('Accuracy Score for model1 = ', accuracy1)
print('Accuracy Score for model2 = ', accuracy2)
print('Accuracy Score for model3 = ', accuracy3)
print('Accuracy Score for average ensemble = ', ensemble_accuracy)

from sklearn.metrics import classification_report
# Print the evaluation metrics for the dataset.
# print(classification_report(Y_test, preds))
print(classification_report(Y_test, ensemble_accuracy))